# Smart Grid Simulation Analysis

This notebook analyzes the results of the Smart Grid Demand-Response Simulator.


### 📚 Step 1: Import Libraries

**Goal**: Prepare the environment.

**Code Explanation**:
- `pandas`: Used for data manipulation (loading CSVs, filtering).
- `plotly.express` & `plotly.graph_objects`: Used for creating interactive charts.
- `make_subplots`: Helper to create charts with multiple axes.

In [1]:
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

### 💾 Step 2: Load Simulation Results

**Goal**: Read the data generated by `main.py`.

**Code Explanation**:
- `pd.read_csv()`: Reads the comma-separated values file.
- `pd.to_datetime()`: Converts the 'ds' column from text to actual Date objects so we can plot them correctly.
- `df.head()`: Shows the first 5 rows to verify data loaded correctly.

In [2]:
# Load Results
df = pd.read_csv('../data/simulation_results.csv')
df['ds'] = pd.to_datetime(df['ds'])
df.head()

,ds,actual_demand,forecast_demand,solar,wind,price,net_load,battery_flow,soc,grid_import,cost
0,2023-01-31 00:00:00,390.368830,404.154810,0.0,37.728313,0.044700,352.640517,50.000000,95.0,402.640517,17.997919
1,2023-01-31 01:00:00,414.331261,407.628392,0.0,150.000000,0.047334,264.331261,5.555556,100.0,269.886816,12.774813
2,2023-01-31 02:00:00,455.923835,418.079845,0.0,150.000000,0.054217,305.923835,0.000000,100.0,305.923835,16.586378
3,2023-01-31 03:00:00,446.679462,433.949402,0.0,150.000000,0.054263,296.679462,0.000000,100.0,296.679462,16.098828
4,2023-01-31 04:00:00,455.527964,453.664334,0.0,150.000000,0.048881,305.527964,0.000000,100.0,305.527964,14.934368


## 1. Demand vs Forecast

Compare actual demand with the forecasted values.


### 📈 Step 3: Analyze Demand vs. Forecast

**Goal**: Check how well our AI model predicted the demand.

**Interpretation**:
- **Blue Line**: The actual electricity usage.
- **Dashed Line**: What our Prophet model predicted.
- **Gap**: The difference is the error. A small gap means the model is good.

In [3]:
fig = go.Figure()
fig.add_trace(go.Scatter(
    x=df['ds'], y=df['actual_demand'], name='Actual Demand'))
fig.add_trace(go.Scatter(
    x=df['ds'], y=df['forecast_demand'], name='Forecast', line=dict(dash='dash')))
fig.update_layout(title='Actual vs Forecasted Demand',
                  xaxis_title='Time', yaxis_title='Power (MW)')
fig.show()

## 2. Renewable Generation

Visualize Solar and Wind generation.


In [4]:
fig = go.Figure()
fig.add_trace(go.Scatter(x=df['ds'], y=df['solar'],
              name='Solar', fill='tozeroy'))
fig.add_trace(go.Scatter(x=df['ds'], y=df['wind'],
              name='Wind', fill='tozeroy'))
fig.update_layout(title='Renewable Generation',
                  xaxis_title='Time', yaxis_title='Power (MW)')
fig.show()

## 3. Grid Import & Battery Dispatch

Analyze how the battery and grid interact to meet demand.


In [5]:
fig = make_subplots(specs=[[{"secondary_y": True}]])

fig.add_trace(go.Scatter(
    x=df['ds'], y=df['grid_import'], name='Grid Import'), secondary_y=False)
fig.add_trace(go.Scatter(x=df['ds'], y=df['battery_flow'],
              name='Battery Flow (+Chg/-Dis)', line=dict(color='green')), secondary_y=False)
fig.add_trace(go.Scatter(x=df['ds'], y=df['soc'], name='State of Charge', line=dict(
    color='purple', dash='dot')), secondary_y=True)

fig.update_layout(title='Grid Import and Battery Usage', xaxis_title='Time')
fig.update_yaxes(title_text="Power (MW)", secondary_y=False)
fig.update_yaxes(title_text="Energy (MWh)", secondary_y=True)
fig.show()

## 4. Cost Analysis

Visualize cost accumulation over time.


In [6]:
df['cumulative_cost'] = df['cost'].cumsum()
fig = px.line(df, x='ds', y='cumulative_cost',
              title='Cumulative Cost Over Time')
fig.show()